In [27]:
# ---------------- IMPORTACIÓN DE LIBRERÍAS ------------------

import csv  # Para leer y escribir archivos CSV (importante para guardar el dataset final)
import os  # Para comprobar rutas y crear carpetas si no existen

from dataclasses import dataclass, asdict  # Para definir estructuras de datos (Piso) y convertirlas en diccionarios
from typing import List, Optional  # Para indicar tipos de datos (listas, campos opcionales, etc.)

from bs4 import BeautifulSoup  # Para analizar el HTML y extraer información de las páginas descargadas


# ----------------- DEFINICIÓN DEL TIPO DE DATO ---------

#Primeor vamos a crear una clase llamada piso, importante pq necesitamos tener una estructura ordenada y limpia, en la que guardaremos los datos de cada piso.
#
@dataclass
class Piso:
    Título: str # El título del anuncio, que será la calle donde esta el piso
    Precio: Optional[str] # El precio del piso 
    Metros_cuadrados: Optional[str] # La superficie en metros cuadrados
    Habitaciones: Optional[str] # Número de habitaciones 
    Barrio: Optional[str] # El barrio o zona 
    url_detalle: Optional[str] # El enlace completo al anuncio en Idealista

# y ahemos definido definir las columnas del dataset, el script buscará dentro del HTML cada anuncio y rellenará una fila con esa estructura.
# Por lo que, ya tenemos la plantilla para guardar los datos de cada piso. 



# ------------ FUNCIONES PARA LEER HTML LOCAL ----
# Aqui vamos a escribir las funciones que se van a utilizar para leer los archivos HTML

# La función "get_soup_from_file" se utiliza para abrir los HTML y prepararlos para que BeautifulSoup los pueda analizar.

def get_soup_from_file(ruta: str) -> BeautifulSoup:
    # Mostramos por pantalla cuál es el archivo que se está analizando
    print(f"Analizando el fichero local: {ruta}")
    # Abrimos el archivo HTML en modo lectura ("r")
    # El parámetro encoding="utf-8" asegura que los acentos y caracteres especiales se lean bien.
    with open(ruta, "r", encoding="utf-8") as f:
        # Leemos todo el contenido del archivo y lo guardamos en la variable "html"
        html = f.read()
    
    # Convertimos el texto HTML en un objeto de BeautifulSoup
    # Esto permite recorrer el contenido del archivo y buscar etiquetas como <article> o <span>.
    return BeautifulSoup(html, "html.parser")



# ----- FUNCIÓN PARA EXTRAER LA INFORMACIÓN DE CADA PISO --------
# Esta función se encarga de sacar los datos de un solo anuncio del HTML.
# Recibe un bloque <article class="item"> (que representa un piso) y devuelve un objeto de tipo Piso.
# Si no consigue encontrar la información básica, devuelve None.

def extraer_piso(card) -> Optional[Piso]:
    """
    Extrae la info de un anuncio dentro de <article class="item ...">
    Si no se encuentra información básica, devuelve None.
    """

    # Se buusca dentro dl anuncio el contenedor con la información principal
    info = card.select_one(".item-info-container")
    #card es ese bloque <article> que representa un piso.
    #.select_one() es una función de BeautifulSoup que sirve para buscar dentro del HTML la primera etiqueta que coincida con un selector CSS
    #".item-info-container" es el nombre de la clase CSS que queremos buscar.
    
    
    #Dentro de cada anuncio, localiza el contenedor principal que guarda la información del piso.
    # Si no se encuentra el contenedor, significa que no hay datos y se devuelve None
    if info is None:
        return None
    
    #Ahora se rellena cada columna de la clase que hemos definido como piso:

    # --------- TÍTULO Y URL DEL ANUNCIO ---------
    # Dentro del bloque de info, buscamos el enlace (<a class="item-link">) que contiene el título y la URL del anuncio en Idealista.
    titulo_el = info.select_one("a.item-link")

    if titulo_el:
        # Se encuentra y se extrae el texto (título del anuncio)
        titulo = titulo_el.get_text(strip=True)
        # Y también se saca la URL (href)
        url_detalle = titulo_el.get("href")
    else:
        # Si no hay enlace, se rellena el título como "Sin título" y la URL como None
        titulo = "Sin título"
        url_detalle = None


    # -------- PRECIO ---------
    # Se busca el elemento que contiene el precio del piso
    precio_el = info.select_one(".item-price")

    # Si existe, se saca su texto (por ejemplo "320.000€"); si no, lo dejamos vacío (None)
    precio = precio_el.get_text(strip=True) if precio_el else None


    # ------- DETALLES DEL PISO (habitaciones y metros cuadrados) ---------
    metros_cuadrados = None
    habitaciones = None

    # En Idealista los detalles suelen estar dentro de <span class="item-detail">
    detalle_spans = info.select(".item-detail-char span.item-detail")

    # Recorremos cada span para ver qué tipo de dato contiene
    for span in detalle_spans:
        txt = span.get_text(strip=True)

        # Si el texto contiene "m²" o "m2", es la superficie del piso
        if "m²" in txt or "m2" in txt:
            # Nos quedamos con el primer número (por ejemplo, de "93 m²" solo "93")
            metros_cuadrados = txt.split()[0]

        # Si contiene "hab", corresponde al número de habitaciones
        if "hab" in txt:
            habitaciones = txt


    # --------BARRIO O ZONA ---------
    # En muchos casos, el barrio aparece dentro del propio título ("Piso en ..., Barrio, Madrid")
    barrio = None
    if "Madrid" in titulo and "," in titulo:
        # Se divide el título por comas
        partes = [p.strip() for p in titulo.split(",")]

        # La penúltima parte suele corresponder al barrio
        if len(partes) >= 2:
            barrio = partes[-2]


    # ------- DEVOLVEMOS EL RESULTADO ---------
    # Creamos un objeto Piso con todos los datos que hemos conseguido
    return Piso(
        Título=titulo,
        Precio=precio,
        Metros_cuadrados=metros_cuadrados,
        Habitaciones=habitaciones,
        Barrio=barrio,
        url_detalle=url_detalle,
    )

# -----------------------------------------------------
# Esta función se encarga de procesar el archivo HTML completo y sacar los pisos que aparecen en ella.
# Para cada anuncio encontrado, llama a la función "extraer_piso" que obtiene los datos individuales.

def extraer_pisos_de_fichero(ruta: str) -> List[Piso]:
    # Se llama a la función anterior para abrir y leer el archivo HTML
    # El resultado es un objeto "soup" que podemos recorrer con BeautifulSoup
    soup = get_soup_from_file(ruta)

    # Se busca dentro del HTML todos los artículos (<article class="item">)
    # Cada artículo representa un piso diferente dentro de la página de Idealista.
    cards = soup.select("section.items-container article.item")

    # Mostramos por pantalla cuántos bloques <article> se han encontrado en ese archivo
    print(f"En {os.path.basename(path)} se han encontrado {len(cards)} bloques <article>")

    # Entonces se crea una lista vacía donde iremos guardando los pisos extraídos
    pisos: List[Piso] = []

    # Y se recorre cada "card" (cada anuncio de piso) dentro del HTML
    for card in cards:
        # Llamamos a la función "extraer_piso" que se encarga de sacar los datos de un solo anuncio
        piso = extraer_piso(card)
        # Si la función devuelve un objeto Piso válido (no None), lo añadimos a la lista
        if piso is not None:
            pisos.append(piso)

    # Mostramos cuántos pisos válidos se han podido extraer realmente
    print(f"  -> Se han podido extraer {len(pisos)} pisos válidos")

    # Devolvemos la lista completa de pisos (cada uno es una fila del dataset)
    return pisos

# ----------------------------------------------------------
# Por último, guardamos todos los datos en un archivo CSV.
# Esta función recibe una lista de pisos (cada uno con sus datos) y los guarda en una tabla con columnas, que será nuestro dataset final.

def guardar_csv(pisos: List[Piso], ruta: str):
    # Primero se comprueba si la lista de pisos está vacía.
    if not pisos:
        print("No hay datos para guardar")
        return

    # Luego, se obtiene los nombres de las columnas a partir del primer objeto Piso.
    
    fieldnames = list(asdict(pisos[0]).keys()) # "asdict" convierte el objeto Piso en un diccionario con clave-valor.

    # Se Crea la carpeta donde se guardará el CSV (si no existe).
    os.makedirs(os.path.dirname(ruta), exist_ok=True)

    # Se abre el archivo CSV en modo escritura ("w").
    # "newline" y "utf-8" aseguran que los acentos y saltos de línea se guarden bien.
    with open(ruta, "w", newline="", encoding="utf-8") as f:
        # Se Crea un escritor de tipo diccionario (usa los nombres de campo como cabecera)
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        # Se escribe la primera línea con los nombres de las columnas
        writer.writeheader()

        # Se recorre todos los pisos y escribimos una línea por cada uno
        for piso in pisos:
            writer.writerow(asdict(piso))  # asdict convierte el objeto Piso en diccionario

    # Se muestra un mensaje indicando que el CSV se ha guardado correctamente
    print(f"CSV guardado en {ruta}")


# ------- EJECUCIÓN: RECORRER TODOS LOS HTML GUARDADOS ---------

ficheros_html = [
    "html/idealista_pagina1.html",
    "html/idealista_pagina2.html",
]

# Aquí se ejecuta la parte final del programa.
# se recorre todos los archivos HTML que hemos descargado de Idealista y se guarda todos los pisos en un único dataset CSV.

# Entpces, se crea una lista vacía donde se iran guardando todos los pisos de todas las páginas
todos_los_pisos: List[Piso] = []

# Se recorre la lista de archivos HTML que se van ha analizar
for ruta in ficheros_html:
    # Se comprueba que el archivo exista antes de procesarlo
    if os.path.exists(ruta):
        # Se extrae los pisos de ese archivo (llamando a la función anterior)
        pisos = extraer_pisos_de_fichero(ruta)

        # Se añaden los pisos obtenidos a la lista general
        todos_los_pisos.extend(pisos)
    else:
        # Si el archivo no existe, mostramos un aviso
        print(f"AVISO: el fichero {ruta} no existe, revisa el nombre o la carpeta")

# Una vez se han procesado todos los HTML, se guardan todos los pisos extraídos en un único archivo CSV
guardar_csv(todos_los_pisos, "dataset/ventaPisosMadridNorte400k_idealista_noviembre_2025.csv")

# y por ultimo, se muestra un mensaje final
print("Proceso completado. Dataset generado correctamente.")



Analizando el fichero local: html/idealista_pagina1.html
En idealista_pagina2.html se han encontrado 30 bloques <article>
  -> Se han podido extraer 30 pisos válidos
Analizando el fichero local: html/idealista_pagina2.html
En idealista_pagina2.html se han encontrado 30 bloques <article>
  -> Se han podido extraer 30 pisos válidos
CSV guardado en dataset/ventaPisosMadridNorte400k_idealista_noviembre_2025.csv
Proceso completado. Dataset generado correctamente.


In [26]:
import pandas as pd

df = pd.read_csv("dataset/ventaPisosMadridNorte400k_idealista_noviembre_2025.csv")
df.head(10)


,Título,Precio,Metros_cuadrados,Habitaciones,Barrio,url_detalle
0,"Piso en Calle de San Dacio, 9, Tres Olivos - V...",300.000€,60,3 hab.,Tres Olivos - Valverde,https://www.idealista.com/inmueble/109393744/
1,"Piso en Valdeacederas, Madrid",389.000€,67,3 hab.,Piso en Valdeacederas,https://www.idealista.com/inmueble/109531024/
2,"Piso en Calle de las Azucenas, Valdeacederas, ...",285.000€,93,2 hab.,Valdeacederas,https://www.idealista.com/inmueble/109794236/
3,"Piso en Tres Olivos - Valverde, Madrid",339.000€,105,3 hab.,Piso en Tres Olivos - Valverde,https://www.idealista.com/inmueble/109725735/
4,"Piso en Calle de Sandalio López, Tres Olivos -...",204.900€,55,3 hab.,Tres Olivos - Valverde,https://www.idealista.com/inmueble/109629837/
5,"Piso en Calle del Plátano, Valdeacederas, Madrid",342.500€,81,3 hab.,Valdeacederas,https://www.idealista.com/inmueble/109290181/
6,"Piso en Ventilla-Almenara, Madrid",350.000€,103,3 hab.,Piso en Ventilla-Almenara,https://www.idealista.com/inmueble/108174078/
7,"Piso en Calle Manuela Mínguez, Valdeacederas, ...",239.900€,54,2 hab.,Valdeacederas,https://www.idealista.com/inmueble/109783444/
8,"Piso en Berruguete, Madrid",305.000€,62,2 hab.,Piso en Berruguete,https://www.idealista.com/inmueble/109458251/
9,"Piso en Tres Olivos - Valverde, Madrid",320.000€,65,3 hab.,Piso en Tres Olivos - Valverde,https://www.idealista.com/inmueble/108746991/


In [28]:
df.shape

(60, 6)

In [30]:
!pip freeze > requirements.txt